# Redownloading NKI JSONS

THe bids validator may have an issue; it's somehow deleting plain text files (json, tsv) files when it runs on large datasets. It may explain how the validator kept showing repeated errors in HBN. It's now shown repeated errors in NKI too.

Fortunately, it seems that running it in series and ensuring the files are read-only may reduce the likelihood of this happening. This notebook replaces files that were previously deleted by the validator.

Essentially, we're going to make repeated calls to `fw download`

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('~/flywheel_curation/RBC/PennLINC/Validation/CUBIC_Curation/NKI_validation.csv')

In [4]:
df.head()

,files,type,severity,description,code,url,subject
0,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,Validation
1,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,Validation
2,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,Validation
3,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,Validation
4,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,Validation


To construct a `fw download` call, we can extract the different parts of the object path using the string in the `files` column. First, getting the subject using `str.extract()`

In [7]:
df['files']

0        /sub-A00027439/ses-BAS1/func/sub-A00027439_ses...
1        /sub-A00027439/ses-BAS1/func/sub-A00027439_ses...
2        /sub-A00027439/ses-FLU1/func/sub-A00027439_ses...
3        /sub-A00027439/ses-FLU1/func/sub-A00027439_ses...
4        /sub-A00027439/ses-FLU1/func/sub-A00027439_ses...
                               ...                        
21392    /sub-A00086748/ses-BAS1/func/sub-A00086748_ses...
21393    /sub-A00086748/ses-BAS1/func/sub-A00086748_ses...
21394    /sub-A00086748/ses-BAS1/func/sub-A00086748_ses...
21395    /sub-A00086748/ses-BAS1/func/sub-A00086748_ses...
21396    /sub-A00086748/ses-BAS1/func/sub-A00086748_ses...
Name: files, Length: 21397, dtype: object

In [9]:
df['files'].str.extract(r'(sub-[a-zA-Z0-9]+)').head()

,0
0,sub-A00027439
1,sub-A00027439
2,sub-A00027439
3,sub-A00027439
4,sub-A00027439


We can do the same for the session labels:

In [11]:
df['files'].str.extract(r'(ses-[a-zA-Z0-9]+)').head()

,0
0,ses-BAS1
1,ses-BAS1
2,ses-FLU1
3,ses-FLU1
4,ses-FLU1


In [12]:
df['subject'] = df['files'].str.extract(r'(sub-[a-zA-Z0-9]+)')
df['session'] = df['files'].str.extract(r'(ses-[a-zA-Z0-9]+)')

In [13]:
df.head()

,files,type,severity,description,code,url,subject,session
0,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1
1,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1
2,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1
3,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1
4,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1


Now the acquisition label:

In [17]:
df['filename'] = df['files'].str.extract(r'([^\/]+$)')
df.head()

,files,type,severity,description,code,url,subject,session,filename
0,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1...
1,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-6...
2,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-BREATHHOLD_acq-140...
3,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-1...
4,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-6...


In [20]:
df['partial_acquisition'] = df['filename'].str.extract(r'[^_]*_[^_]*_(.+?(?=\.))')
df.head()

,files,type,severity,description,code,url,subject,session,filename,partial_acquisition
0,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1...,task-CHECKERBOARD_acq-1400_bold
1,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-6...,task-CHECKERBOARD_acq-645_bold
2,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-BREATHHOLD_acq-140...,task-BREATHHOLD_acq-1400_bold
3,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-1...,task-CHECKERBOARD_acq-1400_bold
4,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-6...,task-CHECKERBOARD_acq-645_bold


In [22]:
df['acquisition'] = df['partial_acquisition'].str.extract(r'(.*(?=_))')
df.head()

,files,type,severity,description,code,url,subject,session,filename,partial_acquisition,acquisition
0,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1...,task-CHECKERBOARD_acq-1400_bold,task-CHECKERBOARD_acq-1400
1,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-6...,task-CHECKERBOARD_acq-645_bold,task-CHECKERBOARD_acq-645
2,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-BREATHHOLD_acq-140...,task-BREATHHOLD_acq-1400_bold,task-BREATHHOLD_acq-1400
3,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-1...,task-CHECKERBOARD_acq-1400_bold,task-CHECKERBOARD_acq-1400
4,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,SLICE_TIMING_NOT_DEFINED,warning,You should define 'SliceTiming' for this file....,13,https://neurostars.org/search?q=SLICE_TIMING_N...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-6...,task-CHECKERBOARD_acq-645_bold,task-CHECKERBOARD_acq-645


Now we can construct the call:

In [38]:
for ind, row in df[:10].iterrows():
    output = ('/'.join([
        'bbl/RBC_NKI',
        row['subject'],
        row['session'],
        row['acquisition']
                       ])
             )
    print(output)

bbl/RBC_NKI/sub-A00027439/ses-BAS1/task-CHECKERBOARD_acq-1400
bbl/RBC_NKI/sub-A00027439/ses-BAS1/task-CHECKERBOARD_acq-645
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-BREATHHOLD_acq-1400
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-CHECKERBOARD_acq-1400
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-CHECKERBOARD_acq-645
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-rest_acq-1400
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-rest_acq-645
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-rest_acq-CAP
bbl/RBC_NKI/sub-A00027439/ses-TRT/task-BREATHHOLD_acq-1400
bbl/RBC_NKI/sub-A00027439/ses-TRT/task-CHECKERBOARD_acq-1400


This is what we will use in our call to get the acquisition object and hopefully download the JSON.

In [40]:
import flywheel

client = flywheel.Client()

In [49]:
for ind, row in df[:10].iterrows():
    output = ('/'.join([
        'bbl/RBC_NKI',
        row['subject'],
        row['session'],
        row['acquisition'],
        'files',
        row['filename']
                       ])
             )
    print(output)

bbl/RBC_NKI/sub-A00027439/ses-BAS1/task-CHECKERBOARD_acq-1400/files/sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1400_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-BAS1/task-CHECKERBOARD_acq-645/files/sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-645_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-BREATHHOLD_acq-1400/files/sub-A00027439_ses-FLU1_task-BREATHHOLD_acq-1400_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-CHECKERBOARD_acq-1400/files/sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-1400_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-CHECKERBOARD_acq-645/files/sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-645_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-rest_acq-1400/files/sub-A00027439_ses-FLU1_task-rest_acq-1400_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-rest_acq-645/files/sub-A00027439_ses-FLU1_task-rest_acq-645_bold.nii.gz
bbl/RBC_NKI/sub-A00027439/ses-FLU1/task-rest_acq-CAP/files/sub-A00027439_ses-FLU1_task-rest_acq-CAP_bold.nii.gz
bbl/RBC_NKI/sub-A000

In [50]:
# it works!
client.lookup('bbl/RBC_NKI/sub-A00027439/ses-BAS1/task-CHECKERBOARD_acq-1400/files/sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1400_bold.nii.gz')

{'classification': {'Intent': ['Functional']},
 'created': datetime.datetime(2019, 12, 20, 18, 11, 3, 787000, tzinfo=tzutc()),
 'deid_log_id': None,
 'hash': 'v0-sha384-c97dfad441bbc6e6dae6e64a4fbb17745f331576aef4ca2618d039e9d8cd089a7281cf3d7cae34fcde697872c71b3a83',
 'id': '17573dde-550d-4f41-82cc-b5201a8f51bb',
 'info': {'BIDS': {'Acq': '1400',
                   'Echo': '',
                   'Filename': 'sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1400_bold.nii.gz',
                   'Folder': 'func',
                   'Path': 'sub-A00027439/ses-BAS1/func',
                   'Rec': '',
                   'Run': '',
                   'Task': 'CHECKERBOARD',
                   'ignore': False,
                   'template': 'task_events_file'},
          'ConversionSoftware': 'dcm2niix',
          'ConversionSoftwareVersion': 'v1.0.20170411 GCC4.3.2',
          'EchoTime': 0.03,
          'EffectiveEchoSpacing': 0.000689998,
          'FlipAngle': 65,
          'ImageType': ['OR

In [53]:
for ind, row in df[:10].iterrows():
    output = ('/'.join([
        'bbl/RBC_NKI',
        row['subject'],
        row['session'],
        row['acquisition'],
        'files',
        row['filename']
                       ])
             )
    
    fi = client.lookup(output)
    print(fi.info)

{'BIDS': {'Task': 'CHECKERBOARD', 'Run': '', 'Filename': 'sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1400_bold.nii.gz', 'ignore': False, 'Acq': '1400', 'template': 'task_events_file', 'Rec': '', 'Path': 'sub-A00027439/ses-BAS1/func', 'Folder': 'func', 'Echo': ''}, 'ConversionSoftware': 'dcm2niix', 'ConversionSoftwareVersion': 'v1.0.20170411 GCC4.3.2', 'EchoTime': 0.03, 'EffectiveEchoSpacing': 0.000689998, 'FlipAngle': 65, 'ImageType': ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC'], 'MagneticFieldStrength': 3, 'Manufacturer': 'Siemens', 'ManufacturersModelName': 'TrioTim', 'PhaseEncodingDirection': 'j-', 'ProcedureStepDescription': 'RESEARCH_colcombe', 'ProtocolName': 'CHECKERBOARD_1400', 'RepetitionTime': 1.4, 'ScanningSequence': 'EP', 'SequenceName': 'epfid2d1_112', 'SequenceVariant': 'SK', 'SeriesDescription': 'CHECKERBOARD_1400', 'SliceTiming': [0, 0.615, 1.23, 0.44, 1.055, 0.2625, 0.8775, 0.0875, 0.7025, 1.3175, 0.5275, 1.1425, 0.3525, 0.965, 0.175, 0.79, 0, 0.615, 1.23, 0.44, 1.05

Now we just need to build the path to the json file:

In [57]:
for ind, row in df[:10].iterrows():
    output = ('/'.join([
        'bbl/RBC_NKI',
        row['subject'],
        row['session'],
        row['acquisition'],
        'files',
        row['filename']
                       ])
             )
    
    fi = client.lookup(output)
    outfile = fi['info']
    del outfile['BIDS']
    path = '~/RBC_RAWDATA/bidsdatasets/NKI' + row['files'].replace('.nii.gz', '.json')
    print(path)

~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-BAS1/func/sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1400_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-BAS1/func/sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-645_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-FLU1/func/sub-A00027439_ses-FLU1_task-BREATHHOLD_acq-1400_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-FLU1/func/sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-1400_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-FLU1/func/sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-645_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-FLU1/func/sub-A00027439_ses-FLU1_task-rest_acq-1400_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-FLU1/func/sub-A00027439_ses-FLU1_task-rest_acq-645_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-FLU1/func/sub-A00027439_ses-FLU1_task-rest_acq-CAP_bold.json
~/RBC_RAWDATA/bidsdatasets/NKI/sub-A00027439/ses-TRT/func/sub-A00027439_ses-TR

In [58]:
outfile

{'AcquisitionMatrixPE': 112,
 'AcquisitionNumber': 1,
 'AcquisitionTime': '11:09:38.485000',
 'BandwidthPerPixelPhaseEncode': 12.94,
 'BaseResolution': 112,
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20180403 GCC5.4.0',
 'DerivedVendorReportedEchoSpacing': 0.000689998,
 'DeviceSerialNumber': '35390',
 'DwellTime': 2.5e-06,
 'EchoTime': 0.03,
 'EffectiveEchoSpacing': 0.000689998,
 'FlipAngle': 65,
 'ImageComments': 'Unaliased_MB4_PE3',
 'ImageOrientationPatientDICOM': [1, -1e-16, 0, 1e-16, 0.999328, -0.0366437],
 'ImageType': ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC'],
 'InPlanePhaseEncodingDirectionDICOM': 'COL',
 'InstitutionAddress': 'Old_Orangeburg_Road_140_Orangeburg_NY_US_10962',
 'InstitutionName': 'Nathan_S_Kline_Institute',
 'InstitutionalDepartmentName': 'Department',
 'MRAcquisitionType': '2D',
 'MagneticFieldStrength': 3,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'TrioTim',
 'Modality': 'MR',
 'MultibandAccelerationFactor': 4,
 'Par

In [82]:
import json
from pathlib import Path

def get_jsons(data, prn=False):
    
    for ind, row in data.iterrows():
        output = ('/'.join([
            'bbl/RBC_NKI',
            row['subject'],
            row['session'],
            row['acquisition'],
            'files',
            row['filename']
                           ])
                 )

        fi = client.lookup(output)
        outfile = fi['info']
        del outfile['BIDS']
        path = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI' + row['files'].replace('.nii.gz', '.json')

        path = Path(path).resolve()
        
        if prn: print(path)
        
        with open(path, 'w') as json_file:
            json.dump(outfile, json_file)

The above should work to write the info to a json file.

## TR Code 10

In [66]:
mask = (df['code'] == 10)
code10 = df[mask]

In [67]:
code10.head()

,files,type,severity,description,code,url,subject,session,filename,partial_acquisition,acquisition
20358,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,REPETITION_TIME_MUST_DEFINE,error,You have to define 'RepetitionTime' for this f...,10,https://neurostars.org/search?q=REPETITION_TIM...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-1...,task-CHECKERBOARD_acq-1400_bold,task-CHECKERBOARD_acq-1400
20359,/sub-A00027439/ses-BAS1/func/sub-A00027439_ses...,REPETITION_TIME_MUST_DEFINE,error,You have to define 'RepetitionTime' for this f...,10,https://neurostars.org/search?q=REPETITION_TIM...,sub-A00027439,ses-BAS1,sub-A00027439_ses-BAS1_task-CHECKERBOARD_acq-6...,task-CHECKERBOARD_acq-645_bold,task-CHECKERBOARD_acq-645
20360,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,REPETITION_TIME_MUST_DEFINE,error,You have to define 'RepetitionTime' for this f...,10,https://neurostars.org/search?q=REPETITION_TIM...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-BREATHHOLD_acq-140...,task-BREATHHOLD_acq-1400_bold,task-BREATHHOLD_acq-1400
20361,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,REPETITION_TIME_MUST_DEFINE,error,You have to define 'RepetitionTime' for this f...,10,https://neurostars.org/search?q=REPETITION_TIM...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-1...,task-CHECKERBOARD_acq-1400_bold,task-CHECKERBOARD_acq-1400
20362,/sub-A00027439/ses-FLU1/func/sub-A00027439_ses...,REPETITION_TIME_MUST_DEFINE,error,You have to define 'RepetitionTime' for this f...,10,https://neurostars.org/search?q=REPETITION_TIM...,sub-A00027439,ses-FLU1,sub-A00027439_ses-FLU1_task-CHECKERBOARD_acq-6...,task-CHECKERBOARD_acq-645_bold,task-CHECKERBOARD_acq-645


In [85]:
get_jsons(code10)

## Task Name Missing Code 50

In [86]:
mask = (df['code'] == 50)
code50 = df[mask]

get_jsons(code50)